In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

from import_ipynb import * 
import Tokenization
import Bert_model 
import collections 
import re 
import json 
import tensorflow as tf  
import codecs 

# sử dụng module tf.flags để định nghĩa các cờ dòng lệnh cho trương trinh . 
# Các cờ dòng lệnh là những tham số có thể được lan truyền vào khi chạy trương trình từ terminal 
# hoặc một môi trường khác . Giúp điều chỉnh các tham số của mô hình mà không cần sửa đổi mã nguồn 
flags = tf.flags 
FLAGS = flags.FLAGS 


In [ ]:
# Thực hiện định nghĩa các cở lệnh với 
# kiểu dtype , tham số , giá trị và mô tả của tham số 

# 1 : định nghĩa một cờ lệnh kiểu string cho tham số input_file gán giá trị = None và mô tả None
flags.DEFINE_string("input_file", None , "")

# 2 : định nghĩa một cò lệnh kiểu string cho tham số output_file gán giá trị = None và mô tả .
flags.DEFINE_string("output_file", None, "")

# 3 : định nghĩa một cò lệnh kiểu string cho tham số layers gán giá trị = None và mô tả .
flags.DEFINE_string("layers", "-1,-2,-3,-4", "")

# 4 : định nghĩa một cò lệnh kiểu string cho tham số bert_config_file gán giá trị = None và mô tả .
flags.DEFINE_string(
    "bert_config_file", None,
    "The config json file corresponding to the pre-trained BERT model. "
    "This specifies the model architecture.")

# 5 : định nghĩa một cò lệnh kiểu integer cho tham số max_seq_length gán giá trị = 128 và mô tả của nó.
flags.DEFINE_integer(
    "max_seq_length", 128,
    "The maximum total input sequence length after WordPiece tokenization. "
    "Sequences longer than this will be truncated, and sequences shorter "
    "than this will be padded.")

# 6 : định nghĩa một cò lệnh kiểu string cho tham số init_checkpoint gán giá trị = None và mô tả của nó.
flags.DEFINE_string(
    "init_checkpoint", None,
    "Initial checkpoint (usually from a pre-trained BERT model).")

# 7 : định nghĩa một cò lệnh kiểu integer cho tham số vocab_file gán giá trị = None và mô tả của nó.
flags.DEFINE_string("vocab_file", None,
                    "The vocabulary file that the BERT model was trained on.")

# 8 : định nghĩa một cò lệnh kiểu bool cho tham số do_lower_cáe gán giá trị = True và mô tả của nó.
flags.DEFINE_bool(
    "do_lower_case", True,
    "Whether to lower case the input text. Should be True for uncased "
    "models and False for cased models.")

# 9 : định nghĩa một cò lệnh kiểu integer cho tham số batch_size gán giá trị = 32 và mô tả của nó.
flags.DEFINE_integer("batch_size", 32, "Batch size for predictions.")

# 10 : định nghĩa một cò lệnh kiểu bool cho tham số use_tpu gán giá trị = False và mô tả của nó.
flags.DEFINE_bool("use_tpu", False, "Whether to use TPU or GPU/CPU.")

# 11 : định nghĩa một cò lệnh kiểu string cho tham số master gán giá trị = None và mô tả của nó.
flags.DEFINE_string("master", None,
                    "If using a TPU, the address of the master.")

# 12 : định nghĩa một cò lệnh kiểu integer cho tham số num_tpu_cores gán giá trị = 8 và mô tả của nó.
flags.DEFINE_integer(
    "num_tpu_cores", 8,
    "Only used if `use_tpu` is True. Total number of TPU cores to use.")

# 13 : định nghĩa một cò lệnh kiểu bool cho tham số  gán giá trị = False và mô tả của nó.
flags.DEFINE_bool(
    "use_one_hot_embeddings", False,
    "If True, tf.one_hot will be used for embedding lookups, otherwise "
    "tf.nn.embedding_lookup will be used. On TPUs, this should be True "
    "since it is much faster.")

In [ ]:
# Xây dựng một lớp là InputExample để định dạng cho  các mẫu đầu vào 
# gồm câu a, câu b và chỉ số id duy nhất của các tokens hoặc của câu 
class InputExample(object):
    # Thiết lập phương thức khởi tạo và định nghĩa các tham số 
    def __init__(self, unique_id, text_a, text_b):
        # gán các tham số cho các thuộc tính 
        self.unique_id = unique_id
        self.text_a = text_a
        self.text_b = text_b

# Tương tự xây dựng một lớp là Input_features để định dnagj cho 
# một tập các đặc trưng riêng lẻ của dữ liệu 
class InputFeatures(object):
    """A single set of features of data."""
    # thiết lập phương thức khởi tạo và định nghĩa các tham số cần sử dụng 
    def __init__(self, unique_id , tokens, input_ids,
                input_mask , input_type_ids):
        # gán các tham số cho các thuộc tính 
        self.unique_id = unique_id
        self.tokens = tokens 
        self.input_ids = input_ids
        self.input_mask = input_mask 
        self.input_type_ids = input_type_ids

    

def input_fn_builder(features,seq_length):
    """Create an 'ionput_fn' clousure to be passed to TPUEstimator.
        Tạo một bao đóng input_fn để chuyển tới TPUestimator.
    """
    # khởi tạo 4 danh sách 
    all_unique_ids = []
    all_input_ids = []
    all_input_mask = []
    all_input_type_ids = []

    # sau đó duyệt qua danh sách chứa các đặc trưng 
    # và thêm các đặc trưng tương ứng từ features vào danh sách tương ứng 
    for feature in features: 
        all_unique_ids.append(feature.unique_id)
        all_input_ids.append(feature.input_ids)
        all_input_mask.append(feature.input_mask)
        all_input_type_ids.append(feature.input_type_ids)

    # xây dựng một phương thức con bên trong input_fb_build 
    def input_fn(params):
        """Chức năng đầu vào thực tế . The actual input function."""
        batch_size = params['batch_size']

        # lấy ra số lượng mẫu = độ dài của dnah sách features
        num_examples = len(features)
        
        # Đây là mục đích Demo và không mở rộng ra các tập dữ liệu lớpk . 
        # không sử dụng Dataset.from_generator() bởi vì nó sử dụng tf.py_func
        # không tương thích với TPU 

        # khởi tạo một biến d và gán nó bằng một định dạng dataset Tf.data.Datase.from_tensor_slice
        # từ các lát cắt tensor [unique_ids , input_ids , input_mask , input_type_ids]
        d = tf.data.Dataset.from_tensor_slices(
            {
                "unique_ids":
                tf.constant(
                    all_unique_ids,
                    shape=[num_examples] , dtype=tf.int32), 
                "input_ids":
                tf.constant(
                    all_input_ids , 
                    shape=[num_examples], dtype= tf.int32), 
                "input_mask":
                tf.constant(
                    all_input_mask,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
                "input_type_ids":
                    tf.constant(
                        all_input_type_ids,
                        shape=[num_examples, seq_length],
                        dtype=tf.int32),
                })
            # chia dữ liệu thành các batch_size mỗi batch gồm batch_size đoạn 
        d = d.batch(batch_size=batch_size , dropout_remainder=False)
        return d 
    


# xây dựng phương thức trả về việc đóng model_fn cho TPUestimator 
def model_fn_builder(bert_config, init_checkpoint, 
                    layer_indexes, use_tpu, 
                    use_one_hot_embeddings):
    """Return 'model_fn' closure for TPUEstimator."""

    # xây dựng một phươnh thức là model_fn trong phương thức model fn 
    def model_fn(features, labels, mode , params):
        """The 'model_fn' for TPUEstimator."""
        # gán cho các biến với các giá trị tương ứng trong danh sách chứa các đặc trưng 
        # features 
        unique_ids = features["unique_ids"]
        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        input_type_ids = features["input_type_ids"]

        # Thiết lập model = Bert_model.Bertmodel 
        model = Bert_model.BertModel(
            config = bert_config, 
            is_training =False , 
            input_ids = input_ids, 
            input_mask = input_mask , 
            token_type_ids = input_type_ids, 
            use_one_hot_embeddings = use_one_hot_embeddings
        )

        # kiểm tra xem chế độ của mô hình có phải là Predict hay không. 
        if mode !=  tf.estimator.ModelKeys.PREDICT:
            # nếu không đưa ra một cảnh báo lỗi 
            raise ValueError("Only PREDICT models are supported: %s" %(model))
        
        # gán tvar bằng các biến có thể được huấn luyện từ mô hình 
        tvars = tf.trainable_variable()
        # Đặt scaffold_fn = None một biến dùng để tạo khung mô hình
        scaffold_fn = None 
        # Lấy bản đồ gán (assigment_map) và tên các biến đã khởi tạo (initialized_variable_names)
        # từ checkpoint , một tệp lưu trữ trạng thái của mô hình 
        (assigment_map , initialized_variable_names) = Bert_model.get_assigment_map_from_checkpoint(
            tvars, init_checkpoint
        )

        # kiểm tra xem mô hình này có đáng ử dụng tpu không 
        if use_tpu: 
            # nếu sử dụng tpu thi định nghĩa một hàm tpu_scaffold trong đó khởi tạo mô hình 
            # từ checkpoint và trả vê 1 đối tượng Scaffold
            def tpu_scaffold():
                tf.train.init_from_checkpoint(init_checkpoint, assigment_map)
                return tf.train.Scaffold()
            
            # gán cho scaffold_fn = kết quả của phương thức tpu_scaffold 
            scaffold_fn = tpu_scaffold
        # trường hợp không sử dụng TPU 
        else: # khởi tạo mô hình trực tiếp từ checkpoint
            tf.train.init_from_checkpoint(init_checkpoint, assigment_map)
        
        # ghi nhật ký và in ra thôngbtin của những biến co thể huấn luyện
        tf.logging.infor("*** Trainable Variables ***")
        # duyệt qua dnh sách các biến có thể huấn luyện từ danh sách tvar
        for var in tvars: 
            # khởi tạo một biến dạng string 
            init_string = ""
            # kiểm tra xem tên của các biến có thể huấn luyện có nằm trong danh sách 
            # tên của các biến được khởi tạo hay không
            if var.name in initialized_variable_names:
                # Gán biến init_string = 1 chuỗi 
                init_string = ", *INIT_FROM_CKPT*"
            # ghi nhật ký các mã thông báo ra màn hình với tên biến , hình dạng và đuôi 
            # cho biết ní được khởi tạo từ checkpoint
            tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                            init_string)
        
        # lấy ra danh sách tất tả các kết quả của lớp encoder từ mô hình 
        all_layers = model.get_all_encoder_layers()

        # tạo một từ điển bao gòm uique_id và accs lớp mã hóa được chọn theo chỉ số tiếp theo 
        predictions = { # từ điển có khóa là unique_id và value là unique_ids 
            "unique_id": unique_ids,
        }

        # lấy ra chỉ số và giá trị tương ững của các phần tử trong danh sách chỉ số lớp 
        # layer_indexes 
        for (i , layer_index) in enumerate(layer_indexes):
            # thêm một cặp khóa và giá trị vào tử điển predict
            # với key = layer_output  theo chỉ số i và values theo giá trị tương ứng của i trong 
            # danh sách đầu ra các lớp mã hóa
            predictions["layer_ouput_%d" %i] = all_layers[layer_index]

        # tạo một đối tương ouput_space 
        output_space = tf.contrib.tpu.TPUEsimatorSpec(
            mode =mode, predictions=predictions, scaffold_fn = scaffold_fn
        )
        # trả về giá trị output_space 
        return output_space
    # trả về model_fn 
    return model_fn 


In [ ]:

# Xây dựng phương thức sử dụng để chuyển đổi các ví dụ thành các đặc trưng cơ bản 
def convert_examples_to_features(examples , seq_length , tokenizer):
    """Load a data file into a list of 'InputBatch'.
        Tải một file dữ liệu trong dnah sách các lô đầu vào .
    """
    # khởi tạo một dnah sách để lưu trữ các đặc trưng 
    features = []
    # duyệt qua các ví dụ lấy ra các chỉ số và giá trị tương ứng 
    for (ex_index, example) in enumerate(examples):
        # lấy ra danh sách token_a từ Tokenizer.tokenize là kết quả của phương thức token hóa 
        # mã thông báo và tìm đến các chỉ số của câu a tương ứng 
        token_a = tokenizer.tokenize(example.text_a)

        # tạo một biến token_b gán = None 
        token_b = None 
        # kiểm tra xem trong accs mẫu ví dụ có tồn tại các token thuộc về danh scahs token_b(câu b)
        if example.token_b : 
            # tương tự lấy ra danh sách token_b từ kết quả của việc token hóa mã thông báo 
            token_b = tokenizer.tokenize(example.token_b)
        
        # kiểm tra xem token_b CÓ tồn tại 
        if token_b: 
            # sửa đổi token_a và token_b tại trỗ sao cho tổng độ dài nhỏ hơn độ dài cụ thể 
            _truncate_seq_pair(token_a , token_b, seq_length - 3)
        
        # trường hợp còn lại tức là không tồn tại token b 
        else: 
            # kiểm tra xem độ dài của token_a có vượt quá độ dài seq_length - 2
            if len(token_b) > seq_length - 2:
                # gán cho tokens a = một chuỗi các phần tử từ 0 -> seq_length - 2
                token_a = token_a[0 :(seq_length -2)]
        
        """ The convention in Bert is:
            Quy ước trong Bert là : 
                (a) For seqence pairs:
                tokens : [CLS] is this jack ##son ##ville ? [SEP] no it is not. [SEP]
                type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1

                (b) For single sentences:
                tokens:   [CLS] the dog is hairy . [SEP]
                type_ids: 0     0   0   0  0     0 0

            
            Trong đó "type_id" được sử dụng để cho biết đây là chuỗi đầu tiên hay chuỗi thứ 2. 
            Các vectoe nhúng cho type = 0 và type = 1 đã được học trong quá trình đào tạo trước
                và được thêm vào vector nhúng từ wordpiece (và vector vị trí) . 
            Điều này không thực sự cần thiết vì các mã thông báo [SEP] phân tách rõ dàng các chuỗi ,
                nhưng nó almf cho mô hình dễ dàng học được khái niệm về trình tự hơn.
        
          
            Cho các nhiệm vụ phân loại vector đầu tiên tương ứng với [CLS] được sử dụng như vector câu. 
            Lưu ý điều này chỉ hợp lý vì toàn bộ mô hình đã đựoc tinh chỉnh .
        """

        # tạo một danh sách tokens [] để lưu trữ danh sách các token và token đặc biệt
        tokens = []
        # và danh sách token_type_ids để lưu trữ các mã hóa one hot 
        input_type_ids = []
        # thêm token [CLS] vào vị trí đầu tiên cho danh sách tokens
        tokens.append("[CLS]")
        # tương tự thêm token 0 vào vị trí đầu tiên của danh sách input_type_ids 
        input_type_ids.append(0)

        # duyệt qua danh sách các phần tử trong danh sách token_a 
        for token in token_a: 
            # thêm các token từ dnah sách token_a vào dnah sách lưu trữ token
            tokens.append(token)
            # và đồng thời thêm các token nhãn = 0 cho danh sách tokens_a 
            input_type_ids.append(0)
        
        # cuối cùng Thêm token [SEP] và tokens cuối cùng cho 2 dnah sách nhằm mụ đích ngăn cách câu 
        tokens.append("[SEP]")
        input_type_ids.append(0)

        # kiểm tra xem có tồn tại danh sách token_b không 
        if token_b: 
            # nếu có ta thực hiện tương tự như làm việc vớ danh sách token_b
            # duyệt qua danh sách các phần tử trong danh sách token_b
            for token in token_b: 
                # thêm các token từ dnah sách token_n vào dnah sách lưu trữ token
                tokens.append(token)
                # và đồng thời thêm các token nhãn = 1 cho danh sách tokens_b 
                input_type_ids.append(1)
            
            # cuối cùng Thêm token [SEP] và tokens cuối cùng cho 2 dnah sách nhằm mụ đích ngăn cách câu 
            tokens.append("[SEP]")
            input_type_ids.append(0)
        
        # chuyển đổi dnah sách tokens thành token_ids dtype = tf.int
        input_ids  = tokenizer.convert_tokens_to_ids(tokens)

        # tạo ma trận pasdding mask 
        # có nhãn là 1 cho các token và 0 cho phần đệm 
        input_mask = [1] * len(input_ids) 

        # sau đó zero_pad đệm 0  theo chiều dài chuỗi 
        while len(input_ids) < seq_length: 
            # thêm các token = 0 vào cuối dnah sách 
            #input_ids 
            input_ids.append(0)
            # input_mask 
            input_mask.append(0)
            # input_type_ids
            input_type_ids.append(0)
        

        # kiểm tra xem độ dài của các danh sách đã đạt đến độ dài của seq_length 
        assert len(input_ids) == seq_length
        assert len(input_mask) == seq_length 
        assert len(input_type_ids) == seq_length 

        # sau đó kiểm tra xem chỉ số hiện tại của examples có < 5 không 
        if ex_index < 5 : 
            # GHI NHẬT KÝ MÃ THÔNG BÁO IN RA MÀN HÌNH THÔNG TIN 
            tf.logging.info("*** Example ***")
            # GHI NHẬT KÝ CÁC ID DUY NHẤT VÀ IN RA MÀN HÌNH THÔNG TIN CỦA CHÚNG 
            tf.logging.info("unique_id: %s" % (example.unique_id))
            # một danh sách các token được tạo ra từ văn bản đầu vào bằng cách sử dụng một hàm 
            # tokenization.printable_text 
            # để chuyển đổi các ký tự không in được thành dạng có thể in được.
            tf.logging.info("tokens: %s" % " ".join(
                [Tokenization.printable_text(x) for x in tokens]))
            # TƯƠNG TỰ NHƯ TRÊN CÁC MÃ IDS CÁCH NHAU BỞI KHOẲNG TRẮNG
            tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            tf.logging.info(
                "input_type_ids: %s" % " ".join([str(x) for x in input_type_ids]))
            
            # thêm các tham số được sử lý là kết quả của phương thức InputFeature
            # vào danh sách features.
            features.append(
                InputFeatures(
                    unique_id=example.unique_id,
                    tokens=tokens,
                    input_ids=input_ids,
                    input_mask=input_mask,
                    input_type_ids=input_type_ids))
    # trả về dnah scahs cac đặc trưng 
    return features



# xây dựng phương thức  __truncate_seq_pair để cắt các cặp câu a, b 
def _truncate_seq_pair(tokens_a, tokens_b ,max_length):
    """Truncates a sequence pair in place to the maximum length.

        Đây là một phương pháp phỏng đoán cơ bản sẽ luôn cắt ngắn chuỗi dài hơn mỗi token tại một 
        bước thời gian. Điều này có nghĩa hơn việc cắt bớt phần trăm số token bằng nhau từ mỗi mã
        thông báo 
    """
    # sử dụng một vòng lặp mô tận 
    while True:
        # kiểm tra xem tổng số độ dài total_length của đoạn = len a  + len b không
        total_length = len(tokens_a) + len(tokens_b)
        # nếu nhỏ hơn không cần cắt 
        if total_length <= max_length:
            break
        # sau đó kiểm tra danh sách tokens_a nếu len(tokens_a) > len(token_b)
        if len(tokens_a) > len(tokens_b):
            # bỏ bớt đi chỉ số cuối của đoạn a
            tokens_a.pop()
        else: # trường hợp còn lại bỏ bớt đi chỉ số cuối của đoạn b
            tokens_b.pop()
        

#  xây dựng phương thức đọc các ví dụ 
def read_examples(input_file):
    """Read a list of 'InputExample's from an input file."""
    # tạo một danh sách examples để lưu trữ các ví dụ 
    examples = []
    # và một danh sách lưu trữ các id duy nhất 
    unique_id = 0
    # đặt phạm vi cho các file được đọc bởi tf.gfile.GFile gán nó cho reader 
    with tf.gfile.GFile(input_file , "r") as reader: 
        # sử dụng một vòng lặp vô tận 
        while True: 
            # gán cho line = kết quả của mã chuyển đổi định dạng thông thường sang unicode 
            # tiêu chuẩn 
            line = Tokenization.convert_to_unicode(reader.readline())
            # kiểm tra xem có tồn tại giá trị line không . 
            if not line : 
                break 
            # chuẩn hóa các dòng văn bản 
            line = line.strip()
            # gán text_a và text_b = None 
            text_a = None 
            text_b = None 
            # tìm kiếm và thay thế danh sách các dongf văn bản với việc so sách các từ với 
            # một biểu thức chính quy nếu = thì bỏ từ hay ký tự đó đi 
            m = re.match(r"^(.*) \|\|\| (.*)$", line)
            # kiểm tra xem m có tồn tại hay không
            if m is None:
                # nếu m không tồn tại thì gán _text_a = line
                text_a = line
            else: # trường hợp còn lại 
                # nhóm dữ liệu thành 2 nhóm 1 và 2 và gán nó cho text_a và text_b 
                text_a = m.group(1)
                text_b = m.group(2)
            # thêm các cí dụ đã được xử lý là kết quả của phương thức InputXample cho một danh sach 
            # Example đã tạo trước đó
            examples.append(
                InputExample(unique_id=unique_id, text_a=text_a, text_b=text_b))
            # sau đó tăng giá trị unique_id lên 1
            unique_id += 1
            # trả về danh sách  example 
        return examples





In [ ]:
def main(_):
    # Thiết lập một mức ghi nhật nhật ký của tensorflow là Infor nghĩa là tensorflow sẻ hiển thik 
    # tất cả các thông tin có nhãn Infor hoă qu trọng hơn. 
    tf.logging._set_verbosity(tf.logging.INFO)
    # tạo một danh sách các chỉ số của các lớp Bert mà muốn trích xuất đặc trưng . 
    # được đingh nghĩa trong mã nguồn FLAGS.layer là 1 chuỗi chứa các chỉ số được phân tách 
    # bởi dấu , 
    layer_indexes = [int(x) for x in FLAGS.layers.split(",")]
    # Tạo một đối tượng Bert Config từ 1 tập Json chứa các thônng số cấu hình của BERT.FLAGS.bert_config
    # LÀ ĐƯỜNG DẪn đến tệp Json 
    bert_config = Bert_model.BertConfig.from_json_file(FLAGS.bert_config_file,)
    # tạo một đối tượng tokenizer dể tách văn bản thành các token . Đối tượng này sử dụng một tệp 
    # từ vựng được chỉ định bởi FLAGS.vocab_file 
    tokenizer = Tokenization.FullTokenizer(
        vocab_file =FLAGS.vocab_file , do_lower_case=FLAGS.do_lower_case
    )

    # Thiết lập một biến is_per_hot để chỉ định các cung cấp dữ liệu cho TPU 
    # để chỉ định cách cung cấp dữ liệu cho TPU 
    is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2 

    # tạo một đối tượng run_config để cấu hình các tham số chạy cho TPU . 
    # đối tượng anyf có 2 tham số master và tpu_config . master là địa chỉ máy chủ TPU , 
    # config là một đối tượng khác chứa các cấu hình cho TPU 
    run_config = tf.contrib.tpu.RunConfig(
        master = FLAGS.master, 
        tpu_config =tf.contrib.tpu.TPUConfig(
            num_shards =FLAGS.num_tpu_cores, 
            per_host_input_for_training=is_per_host
        )
    )

    # đọc một tệp văn bản đầu vào và trả về một đối tượng danh sách các đối tượng InputEXample 
    # mỗi đối tượng trong InputExample chứa một định danh duy nhất và một đoạn văn bản . 
    # FLAGS/input_file là đường dẫn đến tệp văn bản đó 
    examples = read_examples(FLAGS.input_file)

    # chuyển đổi danh sách các ví dụ thành các đặc trưng . Mỗi danh sách InputExample chứa danh sách 
    # các thông tin cần thiết để đưa vào mô hình Bert T, bao gồm các token,
    # các chỉ số của token, các nhãn vị trí, và các mặt nạ
    features = convert_examples_to_features(
        examples=examples , seq_length= FLAGS.max_seq_length , tokenizer=tokenizer,
    )

    # Tạo một từ điển  rỗng để lưu chữ các đối tượng InputDeatures theo mã định danh duy nhất của chúng 
    unique_id_to_feature = {}
    # duyệt qua danh sách các đặc trưng features : 
    for feature in features: 
        # thêm vào từ điển các key và value là id và value_id (feature)
        unique_id_to_feature[feature.unique_id] = feature 
    
    # Thiết lập model_fn = Phương thức model_fn_buil đã được xây dưng trước 
    model_fn = model_fn_builder(
        bert_config=bert_config,
        init_checkpoint=FLAGS.init_checkpoint,
        layer_indexes=layer_indexes,
        use_tpu=FLAGS.use_tpu,
        use_one_hot_embeddings=FLAGS.use_one_hot_embeddings
    )

    #If TPU is not available, this will fall back to normal Estimator on CPU
    # or GPU.
    estimator = tf.contrib.tpu.TPUEstimator(
        use_tpu=FLAGS.use_tpu,
        model_fn=model_fn,
        config=run_config,
        predict_batch_size=FLAGS.batch_size)


    # Tạo một hàm input_feature để cung cấp dữ liệu cho TPU . hàm này sẽ trả về một đối tượng 
    # tf.data.Dataset 
    input_fn = input_fn_builder(
        features=features , seq_length=FLAGS.max_seq_length ,
    )

    # mở một tệp văn bản đầu ra với chế độ ghi và mã hóa UTF8 tệp này se chứa các đặc trưng được tríc 
    # xuất từ mô hình BERT.FLAGS.output_file  là đường dẫn đến tập đó đuôi tệp.JSON 
    with codecs.getwriter("utf-8")(tf.gfile.Open(FLAGS.output_file, "w")) as writer: 

        # duyệtq au các kết quả được trả về bởi phương thức predict của đối tượng estimator
        # phương thức predict sẽ chạy mô hình bert 
        for result in estimator.predict(input_fn , yield_single_examples=True ):
            # lấy giá trị của khóa unique_id trong từ điển result và chuyển nó thành một số nguyên . 
            # dây là định dnah duy nhất của một đoạn avwn bản đầu vào
            unique_id = int(result["unique_id"])
            # lấy đối tượng InputFeature tương ứng với định danh duy nhất từ từ điển unique_id_to_f
            feature = unique_id_to_feature[unique_id]
            # tạo một từ điển output json là một từ điển có thứ tự 
            output_json = collections.OrderedDict()
            # Thêm một khóa index_line vào từ điển output_json với giá trị định dnah duy nhất của đoan văn bản 
            output_json["linex_index"] = unique_id

            # tạo một danh sách all_feature để lưu trữ các thông tin về các token và các lớp trong mô hình BERT.
            all_features = []
            # duyệt qua 1 danh sách các tokens trong đối tương feature 
            for (i , token) in enumerate(feature.tokens):
                # tạo một danh scags all_layer để lưu trữ các lớp và thông tin accs lớp trong Bert 
                all_layers = []
                # Duyệt qua danh sách các chỉ số của các lớp trong mô hình BERT, 
                # với chỉ số j và giá trị layer_index
                for (j , layer_index) in enumerate(layer_indexes):
                    #  Lấy giá trị của khóa “layer_output_%d” trong từ điển result, với %d là giá trị của j.
                    layer_output = result["layer_output_%d" % j]
                    # tạo một đối tượng layers là một từ điển có thứ tự 
                    layers = collections.OrderedDict()
                    # : Thêm một khóa “index” vào từ điển layers, 
                    # với giá trị là chỉ số của lớp thứ j, ví dụ 0, 1, 2, 3.    
                    layers["index"] = layer_index
                    # Thêm một khóa “values” vào từ điển layers, với giá trị là một danh sách các số thực
                    layers["values"] = [
                        round(float(x), 6) for x in layer_output[i:(i + 1)].flat
                    ]
                    # hêm đối tượng layers vào danh sách all_layers.
                    all_layers.append(layers)
                # tạo một từ điển có thứ tự và gán nó cho features 
                features = collections.OrderedDict()
                # : Thêm một khóa “token” vào từ điển features, với giá trị là token thứ i, ví dụ “Hello”.
                features["token"] = token
                #  Thêm một khóa “layers” vào từ điển features, với giá trị là danh sách all_layers, 
                # chứa các thông tin về các lớp trong mô hình BERT cho token thứ i
                features["layers"] = all_layers
                # Thêm đối tượng features vào danh sách all_features.
                all_features.append(features)
            #: Thêm một khóa “features” vào từ điển output_json, với giá trị là danh sách all_features,
            #  chứa các thông tin về các token và các lớp trong mô hình BERT ch
            output_json["features"] = all_features
            # : Ghi đối tượng output_json vào tệp văn bản đầu ra, sử dụng phương thức dumps của thư viện
            # json để chuyển đổi từ điển thành chuỗi JSON. 
            # Thêm ký tự xuống dòng “\n” để phân tách các đoạn văn bản khác nhau.
            writer.write(json.dumps(output_json) + "\n")


# phần cuối của chương trình sử dụng mô hình BERT để trích xuất các đặc trưng của văn bản
if __name__ == "__main__":
  #  Đánh dấu rằng tham số “input_file” là bắt buộc phải có khi chạy chương trình.
  flags.mark_flag_as_required("input_file")
  # Đánh dấu rằng tham số “vocab_file” là bắt buộc phải có khi chạy chương trình. 
  # Tham số này là đường dẫn đến tệp từ vựng của mô hình BERT,
  flags.mark_flag_as_required("vocab_file")
  #  Đánh dấu rằng tham số “bert_config_file” là bắt buộc phải có khi chạy chương trình. 
  # Tham số này là đường dẫn đến tệp JSON chứa các thông số cấu hình của mô hình BERT
  flags.mark_flag_as_required("bert_config_file")
  #  Đánh dấu rằng tham số “init_checkpoint” là bắt buộc phải có khi chạy chương trình. Tham 
  # số này là đường dẫn đến điểm kiểm tra được lưu trữ của mô hình BERT, ví dụ “bert_model.ckpt”
  flags.mark_flag_as_required("init_checkpoint")
  # Đánh dấu rằng tham số “output_file” là bắt buộc phải có khi chạy chương trình.
  #  Tham số này là đường dẫn đến tệp văn bản đầu ra
  flags.mark_flag_as_required("output_file")
  # : Chạy chương trình bằng cách gọi hàm main() được định nghĩa ở đâu đó trong mã nguồn. 
  # Hàm main() sẽ tạo một đối tượng estimator để huấn luyện và suy luận mô hình BERT,
  #  sử dụng các tham số được truyền vào từ dòng lệnh
  tf.app.run()
